In [25]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import os


def work_space(path):
    while True:
        if os.path.split(path)[1] != "Programs":
            path = os.path.split(path)[0]
        else:
            return path.replace("\\", "/")

work_dir = work_space(os.getcwd())
data_path = work_dir + "/Data/Pandas/"
path_to_save = work_dir + "/Hub/EventData/"

folders = ["Background", "BH", "Sphaleron"]
stuffs = ["electron", "jet", "MET", "muon", "photon", "tau"]
file_amounts = [2, 18, 3]


from FilesFunc import files
folder_list, filenames_list = files(data_path = data_path, folders = folders, stuffs = stuffs, file_amounts = file_amounts)


data_variables = ["eta", "phi", "PT", "jmass", "ntrk", "btag", "hadem", "event#"]
from DataFunc import data


def unpacker(folder_data, new_folder_data):
    for nested_list in folder_data:
        if type(nested_list) == list:
            unpacker(nested_list, new_folder_data)
        else:
            new_folder_data.append(nested_list)
    folder_data = new_folder_data
    return folder_data


def event_counts(event_data):
    counts = [0 for stuff in stuffs]
    particle_name_index = 0
    for event in event_data:
        for stuff_index, stuff in enumerate(stuffs):
            count = len([0 for particle in event if particle[particle_name_index] == stuff])
            counts[stuff_index] = count if counts[stuff_index] < count else counts[stuff_index]
    return counts


event_data = unpacker([data(file_data, stuffs, data_variables) for folder_files in folder_list for file_data in folder_files], [])
counts = event_counts(event_data)


for folder_files, filenames, folder in zip(folder_list, filenames_list, folders):
    for file_data, filename in zip(folder_files, filenames):
        event_data = data(file_data, stuffs, data_variables)
        dict_data = {i: [[np.nan for data_variable in data_variables] for particle in range(np.sum(counts))] for i in range(len(event_data))}

        for event_index, event in enumerate(event_data):
            event_len = len(event)
            for stuff_index, stuff in enumerate(stuffs):
                count = counts[stuff_index]
                particle_name_index = 0
                particles = [particle for particle in event if particle[particle_name_index] == stuff]
                for particle_index, particle in enumerate(particles):
                    particle_index += int(np.sum(counts[:stuff_index]))
                    dict_data[event_index][particle_index] = particle

        data_df = list(dict_data.values())
        particles_df = np.array([stuff + str(i) for stuff_index, stuff in enumerate(stuffs) for i in range(counts[stuff_index])])
        dataframe = pd.DataFrame(data_df, columns = particles_df).assign(datasets = [folders.index(folder) for event in range(len(event_data))])
        
        filename = path_to_save + folder + "/" + filename + ".csv"
        dataframe.to_csv(filename)